# Working with the TAPE Ensemble

When working with many lightcurves, the `TAPE` `Ensemble` object serves as a singular interface for storing, filtering, and analyzing timeseries data.

Let's create a small example `Ensemble` from the Stripe 82 RRLyrae dataset.

In [ ]:
from tape.ensemble import Ensemble

ens = Ensemble().from_dataset("s82_rrlyrae", sort=True)

## The Object and Source Frames
The `Ensemble` is an interface for tracking and manipulating a collection of dataframes. When first intialized, an `Ensemble` tracks two tables (though additonal tables can be added the `Ensemble`), the "Object dataframe" and the "Source dataframe".

This borrows from the Rubin Observatories object-source convention, where object denotes a given astronomical object and source is the collection of measurements of that object. Essentially, the Object frame stores one-off information about objects, and the source frame stores the available time-domain data.

The dataframes tracked by the `Ensemble` are `EnsembleFrames` (of which the Source and Object tables are special cases).

<div style="text-align: center;">
    <img src="../_static/tutorial_images/ensemble_api_structure.png" alt="Visualization of the TAPE Ensemble API structure" width="50%"/>
</div>
You can access the individual Source and Object dataframes with `Ensemble.source` and `Ensemble.object` respectively.

In [ ]:
ens.source

In [ ]:
ens.object

## Dask and "Lazy Evaluation"

`TAPE` is built on top of [Dask](https://github.com/dask/dask), a framework for flexible parallelization and data analytics.

`TAPE`'s `EnsembleFrame` objects inherit from [Dask dataframes](https://docs.dask.org/en/stable/dataframe.html#dask-dataframe) and support most `Dask` operations.

Note that Source and Object tables are `SourceFrames` and `ObjectFrames` respectively, which are special cases of `EnsembleFrames`.
<div style="text-align: center;">
    <img src="../_static/tutorial_images/ensemble_frame_hierarchy.png" alt="EnsembleFrame class hierarchy" width="50%"/>
</div>

An important feature of `Dask` is that it evaluates code "lazily". This means that many operations are not executed when the line of code is run, but instead are added to a scheduler to be executed when the result is actually needed.

As an example:

In [ ]:
ens.source  # We have not actually loaded any data into memory

When accessing the Source dataframe above, we only saw an empty dataframe with some high-level information about its schema. To explicitly bring the data into memory, we must run a `compute()` command on the dataframe.

In [ ]:
ens.source.compute()  # Compute lets Dask know we're ready to bring the data into memory

With this `compute()`, we returned a populated dataframe. 

Many workflows in `TAPE` use this `Dask` paradigm and will look like a series of lazily evaluated commands that are chained together and then executed with a `compute()` call at the end of the workflow.

## Updating an Ensemble's Frames

As discussed above, an `Ensemble` is a manager of `EnsembleFrame` objects (and `Ensemble.source` and `Ensemble.object` are special cases). When performing most operations on one of these tables, the results are not automatically updated to the `Ensemble`.

Here we filter `Ensemble.source` by its flux column (see more examples of filtering using these `Dask`/`Pandas` style operations in [Common Data Operations with TAPE](https://tape.readthedocs.io/en/latest/tutorials/common_data_operations.html#Filtering)), but note there were no changes to the rows of `Ensemble.source`.

In [ ]:
filtered_src = ens.source.query(f"{ens._flux_col} > 15")

print(len(filtered_src))
print(len(ens.source))

Most dataframe operations will return a result frame that is not yet tracked by the `Ensemble`. When modifying the views of a dataframe tracked by the `Ensemble`, we can update the Source or Object frames to use the updated result frame by calling

`Ensemble.update_frame(filtered_src)`

Or alternately:

`filtered_src.update_ensemble()`

In [ ]:
# Now apply the filtered result to the Source frame.
filtered_src.update_ensemble()

ens.source.compute()

The Source frame now has the number of rows we saw when inspecting the filtered result above.

Note that the above is still a series of lazy operations that will not be fully evaluated until an operation such as `compute()`. So a call to `update_ensemble()` will not yet alter or move any underlying data.

# Storing and Accessing Result Frames
The `Ensemble` provides a powerful batching interface, `Ensemble.batch()`, to perform analysis functions in parallel across your lightcurves.

For the below example, we use the included suite of analysis functions to apply `tape.analysis.calc_stetson_J` on our dataset. (For more info on `Ensemble.batch()`, including providing your own custom functions, see the [Ensemble Batch Showcase](https://tape.readthedocs.io/en/latest/tutorials/batch_showcase.html#) )

In [ ]:
# using TAPE analysis functions
from tape.analysis import calc_stetson_J

res = ens.batch(calc_stetson_J)
res

Note for the above `batch` operation, we also printed output in the form of

```
    "Using generated label, {label}, for a batch result."
```

In addition to the Source and Object frames, the `Ensemble` may track other frames as well, accessed by either generated or user-provided labels.

We can access a saved frame with `Ensemble.select_frame(label)`

In [ ]:
ens.select_frame("result_1").compute()

`Ensemble.batch` has an optional `label` argument that will store the result with a user-provided label.

In [ ]:
res = ens.batch(calc_stetson_J, label="stetson_j")

ens.select_frame("stetson_j").compute()

Likewise we can rename a frame with with a new label, and drop the original frame.

In [ ]:
ens.add_frame(ens.select_frame("stetson_j"), "stetson_j_result_1")  # Add result under new label
ens.drop_frame("stetson_j")  # Drop original label

ens.select_frame("stetson_j_result_1").compute()

We can also add our own frames with `Ensemble.add_frame(frame, label)`. For instance, we can copy this result and add it to a new frame for the `Ensemble` to track as well.

In [ ]:
ens.add_frame(res.copy(), "new_res")
ens.select_frame("new_res")

Finally we can also drop frames we are no longer interested in having the `Ensemble` track.

In [ ]:
ens.drop_frame("new_res")

try:
    ens.select_frame("new_res")  # This should result in a KeyError since the frame has been dropped.
except Exception as e:
    print("As expected, the frame 'new_res' was dropped.\n", str(e))

# Keeping the Object and Source Tables in Sync

The `TAPE` `Ensemble` attempts to lazily "sync" the Object and Source tables such that:

* If a series of operations removes all lightcurves for a particular object from the Source table, we will lazily remove that object from the Object table.
* If a series of operations removes an object from the Object table, we will lazily remove all light curves for that object from the Source table.

As an example, let's filter the Object table only for objects of type 'ab'. This operation marks the result table as `dirty` indicating to the `Ensemble` that if used as part of a result computation, it should check if the Object and Source tables are synced. 

Note that because we have not called `update_ensemble()` the `Ensemble` is still using the original Object table which is **not** marked `dirty`.


In [ ]:
type_ab_only = ens.object.query("type == 'ab'")

print("Object table is dirty: ", str(ens.object.is_dirty()))
print("ddf_only is dirty: ", str(type_ab_only.is_dirty()))
type_ab_only.compute()

Now let's update the `Ensemble`'s Object table. We can see that the Object table is now considered "dirty" so a sync between the Source and Object tables will be triggered by computing an `Ensemble.batch()` operation. 

As part of the sync the Source table has been modified to drop all sources for objects not with types other than 'ab'. This is reflected both in the `Ensemble.batch()` result output and in the reduced number of rows in the Source table.

In [ ]:
type_ab_only.update_ensemble()
print("Updated object table is now dirty: " + str(ens.object.is_dirty()))

print("Length of the Source table before the batch operation: " + str(len(ens.source)))
res = ens.batch(calc_stetson_J).compute()
print("Post-computation object table is now dirty: " + str(ens.object.is_dirty()))
print("Length of the Source table after the batch operation: " + str(len(ens.source)))
res

To summarize:

* An operation that alters a frame marks that frame as "dirty"
* Such an operation on `Ensemble.source` or `Ensemble.object` won't cause a sync between the Source and Object if the output frame has not been stored back to either `Ensemble.source` or `Ensemble.object` respectively. This is usually done by a call to `EnsembleFrame.update_ensemble()`
* Syncs are done lazily such that even when the Object and/or Source frames are "dirty", a sync between tables won't be triggered until a relevant computation yields an observable output, such as `batch(..., compute=True)` or `Ensemble.source.merge(result).compute()`